In [ ]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import mixed_precision
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau, 
                                      ModelCheckpoint, TerminateOnNaN)
from tensorflow.keras.layers import (Input, ConvLSTM2D, MaxPooling3D, 
                                   Dropout, Dense, GlobalAveragePooling3D,
                                   TimeDistributed, MultiHeadAttention,
                                   Reshape, GlobalMaxPooling1D,LayerNormalization)

import tensorflow as tf
from classes import CLASSES_LIST, IMAGE_HEIGHT, IMAGE_WIDTH, SEQUENCE_LENGTH, BATCH_SIZE, DATASET_DIR


In [ ]:
# GPU Configuration for TensorFlow (Optimized for GTX 1650 Max-Q)
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

# Configure GPU memory growth to avoid OOM errors on 4GB VRAM
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth for GTX 1650 Max-Q
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Set memory limit to ~3.5GB to leave room for system
        tf.config.experimental.set_memory_limit(gpus[0], 3584)  # 3.5GB in MB
        print(f"Using GTX 1650 Max-Q - Memory limit set to 3.5GB")
    except RuntimeError as e:
        print(f"GPU configuration error: {e}")
        print("Falling back to default configuration")
else:
    print("No GPU found, using CPU")


In [ ]:
# Load saved features
def load_features():
    features, labels = [], []
    for class_idx, class_name in enumerate(CLASSES_LIST):
    # enumerate - a function for iterating through the elements and returning them along with their indexes 
        feature_dir = os.path.join("features", class_name)
        for feature_file in os.listdir(feature_dir):
            feature_path = os.path.join(feature_dir, feature_file)
            feat = np.load(feature_path, mmap_mode='r')
            # mmap_mode - a parameter telling to use momory-mapped mode for uploading data
            # saving memory
            features.append(feat)
            labels.append(class_idx)
    
    return np.array(features), to_categorical(labels)


In [ ]:
# Used functional model style for compatibility with MultiHeadAttention,
# which allows a model to focus on different parts of the input sequence simultaneously,
# capturing multiple types of relationships. 

def build_hybrid_model():
    # Input layer: features from EfficientNetB0
    inputs = Input(shape=(SEQUENCE_LENGTH, 1280), name='input_layer')
    
    # Transform the entry form: (SEQUENCE_LENGTH, 1, 1, 1280)
    x = Reshape((SEQUENCE_LENGTH, 1, 1, 1280), name='reshape_for_lstm')(inputs)

    # ConvLSTM2D layer to capture time dependencies
    x = ConvLSTM2D(
        filters=70,
        kernel_size=(1, 1),
        activation='tanh',
        recurrent_activation='sigmoid',
        recurrent_dropout=0.2,
        return_sequences=True,
        name='convlstm2d'
    )(x)

    # Unfolding spatial dimensions before the attention method
    x = Reshape((SEQUENCE_LENGTH, -1), name='reshape_for_attention')(x)

    # Normalizes each token’s embedding vector across its features.
    # Prevents exploding/vanishing activations and prepares for the attention layer.
    norm_x = LayerNormalization(name='pre_attn_norm')(x)

    attention_output = MultiHeadAttention(
        num_heads=4,
        key_dim=64,
        name='multihead_attention'
    )(norm_x, norm_x)  # query = key = value = norm_x
    # num_heads=4 - number of parallel heads of attention
    # key_dim=128 - The dimension of the vectors of the key (Key) and query (Query) for each head. Affects the detail of attention
    """
        Query searches for connections with Key.
        Value returns the context for the Query(for example, that the action is related to something)
    """

    # Dropout for regularization after attention
    x = Dropout(0.2, name='dropout_after_attention')(attention_output)

    # Averaging along the time axis
    x = GlobalMaxPooling1D(name='global_avg_pool')(x)

    # Fully connected layer with 'ReLU' activation
    x = Dense(128, activation='relu', name='dense_relu')(x)

    # Dropout after a fully connected layer
    x = Dropout(0.4, name='dropout_after_dense')(x)

    # Output layer with 'softmax' for classification
    outputs = Dense(len(CLASSES_LIST), activation='softmax', dtype='float32', name='output_softmax')(x)

    # Building a model
    model = Model(inputs=inputs, outputs=outputs, name='Hybrid_Attention_Model')

    return model


In [ ]:
# This class implements a batch generator 
# with the ability to augment data "on the fly" during model training,
# to avoid overfitting.
# Still need to use this method very carefully.

class AugmentedDataGenerator(Sequence):
# Sequence -  makes our generator compatible with model.fit() and it allows you to use multithreading.
    
    def __init__(self, x_data, y_data, batch_size, augment=False):
        # Save the input data and settings
        self.x = x_data                     
        self.y = y_data                     
        self.batch_size = batch_size        
        self.augment = augment 
        self.indices = np.arange(len(x_data))

    def __len__(self):
        # Returns the number of batches per epoch from the entire dataset to be generated.
        # ceil - rounding it up to a higher value
        return int(np.ceil(len(self.x) / self.batch_size))
        # Divides the amount of data (len(self.x)) by the size of the batch (self.batch_size).
    
    def __getitem__(self, idx):
    # This method is called when requesting the next batch during training.

        batch_indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]  # Pulling out the indexes of the current batch
        batch_x = self.x[batch_indices]   # selection of features by indexes
        batch_y = self.y[batch_indices]   # selection of class labels by indexes
        
        if self.augment:
            batch_x = self._augment_batch(batch_x)  # apply augmentation if enabled
            
        return batch_x, batch_y  # returns finished batch (X and y)

    def _augment_batch(self, batch):
        # Performs data augmentation,
        # for example, randomly flips sequences 
        # (in this case, along the time axis).
        augmented = []
        for seq in batch:
            if random.random() > 0.5:
            # Выбираем рандомно
                seq = np.flip(seq, axis=0)  # Временной разворот
            augmented.append(seq)
        return np.array(augmented)
    



In [ ]:
def train_model():
    print("Loading features...")
    features, labels = load_features()
    # Split into training and test parts.
    x_train, x_test, y_train, y_test = train_test_split(
        features, labels, test_size=0.1, stratify=labels, random_state=42)
    
    # Build model
    with tf.device('/GPU:0'):
        model = build_hybrid_model()
    optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=True)
    model.compile(optimizer=optimizer,
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    
    # Callbacks
    callbacks = [
        EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
        # If val_accuracy does not increase for 6 epochs, then the training process stops
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6),
        # The same item as EarlyStopping, but for val_loss, only here learning_rate is reduced by multiplying by 'factor'.
        # Prevents overfitting
        TerminateOnNaN() # Stop training at one of the parameters = NaN
    ]
    
    # Data augmentation
    train_gen = AugmentedDataGenerator(x_train, y_train, BATCH_SIZE, augment=True)
    val_gen = AugmentedDataGenerator(x_test, y_test, BATCH_SIZE)
    
    # Train model
    print("Starting training...")
    history = model.fit(
        train_gen,
        epochs=50,
        validation_data=val_gen,
        # tests
        callbacks=callbacks,
        verbose=1
        # verbose - shows 1 training scale for each epoch
    )
    
    return model, history


In [ ]:
# Uploading a model for further training, if it exists
# or
# Creating it
if os.path.exists("trained_model_.keras"):
    model = load_model("trained_model_.keras")
    print("The existing model has been loaded")
else:
    model, history = train_model()

    # Building charts
    train_acc = history.history['accuracy']
    test_acc = history.history['val_accuracy']
    train_loss = history.history['loss']
    test_loss = history.history['val_loss']

    def plot_metric(metric1, metric2, plot_n):
        if len(metric1) != len(metric2):
            print(f"! Warning: The metric lengths do not match! Metric1: {len(metric1)}, Metric2: {len(metric2)}\n")
            min_len = min(len(metric1), len(metric2))
            metric1 = metric1[:min_len]
            metric2 = metric2[:min_len]
            
        epochs = range(len(metric1))

        plt.plot(epochs, metric1, 'r', label=f'{plot_n} (train)')
        plt.plot(epochs, metric2, 'b', label=f'{plot_n} (validation)')
        plt.title(f'Training and Validation {plot_n}')
        plt.legend()
        plt.show()

    plot_metric(train_acc, test_acc, 'Accuracy')
    plot_metric(train_loss, test_loss, 'Loss')


    model.save("trained_model_.keras")
    print("№ The model was successfully created and saved")
    
